# Doc2Vec experiment

Sadly doc2 vec seems to work well only in a sentence to vector fashion might be a good idea to just map a set of documents to a set of sentences and on top of that just use the coherency based approach.

Additionally providing two identically documents I get different vectors thus there is a bug in my Class. Might have to code the coherency based approach on top of gensims sent2vec or doc2vec.

In [2]:
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedLineDocument
from gensim.models.doc2vec import Doc2Vec
from gensim import utils
import numpy as np

In [19]:
from itertools import chain
class LabeledDocs(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                # print fin.read().strip(r"\n")
                yield LabeledSentence(utils.to_unicode(fin.read()).split(),
                                      [prefix])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                #print fin, fin.read()
                self.sentences.append(LabeledSentence(utils.to_unicode(fin.read()).split(),
                                                     [prefix ]))
        return self.sentences
    


In [20]:
inp = LabeledDocs({"play":"play", "play3":"play3"})

In [21]:
model = Doc2Vec(size=20, window=8, min_count=2, workers=1, alpha=0.025, min_alpha=0.025, batch_words=30)


In [22]:
model.build_vocab(inp.to_array())

In [23]:
for epoch in range(10):
    model.train(inp)

In [24]:
import numpy as np
np.sum(model.docvecs["play"])

0.018146381

In [25]:
np.sum(model.docvecs["play3"])

0.098417841

In [26]:
model.docvecs["play3"] == model.docvecs["play"]  # why is this the case if the docs are identical

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False], dtype=bool)

In [23]:
# play 4 has 3 identical paragraphs
doc = TaggedLineDocument("play4")

In [24]:
model = Doc2Vec(size=20, window=8, min_count=2, workers=1, alpha=0.025, min_alpha=0.025, batch_words=1)
model.build_vocab(doc)

In [27]:
# forcing these sentences to give the same vectors besides the different random seeds per paragraph
for epoch in range(100000):
    model.train(doc)

In [29]:
# the more epochs the more itentical setences become
np.array(model.docvecs)

array([[-0.51845229,  0.37670702,  0.56653804, -0.63391662,  0.0713364 ,
         0.16353321, -0.0431617 , -0.24531184,  0.40009505, -0.13420418,
        -0.1995839 , -0.00493006, -0.40535817, -0.02384069,  0.30983743,
         0.5252257 , -0.54618013,  0.09623739,  0.03955276,  0.1998809 ],
       [-0.54277772,  0.39374539,  0.59901023, -0.67358571,  0.07437911,
         0.17486122, -0.04132501, -0.25859055,  0.41882136, -0.14497253,
        -0.20826729, -0.00561933, -0.42267647, -0.0249706 ,  0.3268317 ,
         0.55333692, -0.58275348,  0.09910247,  0.04739791,  0.21325673],
       [-0.54715413,  0.39686921,  0.60437012, -0.67958081,  0.07488864,
         0.17662121, -0.04153084, -0.26062968,  0.42223096, -0.14651592,
        -0.20983626, -0.00574473, -0.42606226, -0.02526709,  0.32961735,
         0.55789196, -0.58806503,  0.09967168,  0.04818159,  0.21543086]], dtype=float32)

In [14]:
np.array(model.docvecs).shape

(5, 20)